# Analyse av Runkeeper-data

### Flere ideer

https://ocefpaf.github.io/python4oceanographers/blog/2014/08/18/gpx/

https://towardsdatascience.com/how-tracking-apps-analyse-your-gps-data-a-hands-on-tutorial-in-python-756d4db6715d

- Vis hvor mye jeg har løpt per år/mnd

- Ha forskjellig farge på hver tur etter hvor langt jeg løp:

https://medium.com/@madhuramiah/geographic-plotting-with-python-folium-2f235cc167b7

https://betterfigures.org/2015/06/23/picking-a-colour-scale-for-scientific-graphics/

### Importerer pakker

In [2]:
import gpxpy
import matplotlib.pyplot as plt
import datetime
from geopy import distance
from math import sqrt, floor
import numpy as np
import pandas as pd
import haversine
import folium
from folium.plugins import HeatMap
import glob
import os

### Laster og behandler alle gpx-datene med en funksjon

https://ocefpaf.github.io/python4oceanographers/blog/2014/08/18/gpx/

In [3]:
def load_run_data(filter=""):
    gpx_files = glob.glob('Data/' + filter + '*.gpx')    
    run_data = []
    for file_idx, gpx_file in enumerate(gpx_files): 
        gpx = gpxpy.parse(open(gpx_file, 'r'))
        # Loop through tracks
        for track_idx, track in enumerate(gpx.tracks):
            track_name = track.name
            track_time = track.get_time_bounds().start_time
            track_length = track.length_3d()
            track_duration = track.get_duration()
            track_speed = track.get_moving_data().max_speed
            
            for seg_idx, segment in enumerate(track.segments):
                segment_length = segment.length_3d()
                for point_idx, point in enumerate(segment.points):
                    run_data.append([file_idx, os.path.basename(gpx_file), track_idx, track_name, 
                                     track_time, track_length, track_duration, track_speed, 
                                     seg_idx, segment_length, point.time, point.latitude, 
                                     point.longitude, point.elevation, segment.get_speed(point_idx)])
    return run_data

### Lager DataFrame av gpx-datene

In [13]:
data = load_run_data(filter="")
df = pd.DataFrame(data, columns=['File_Index', 'File_Name', 'Index', 'Name',
                              'Time', 'Length', 'Duration', 'Max_Speed',
                              'Segment_Index', 'Segment_Length', 'Point_Time', 'Point_Latitude',
                              'Point_Longitude', 'Point_Elevation', 'Point_Speed'])

df.tail()

,File_Index,File_Name,Index,Name,Time,Length,Duration,Max_Speed,Segment_Index,Segment_Length,Point_Time,Point_Latitude,Point_Longitude,Point_Elevation,Point_Speed
152743,335,2019-12-02-161113.gpx,0,Cross-Country Skiing 12/2/19 4:11 pm,2019-12-02 15:11:13+00:00,4250.317829,2222,4.784425,0,4250.317829,2019-12-02 15:47:29+00:00,60.200203,12.006000,202.4,1.506337
152744,335,2019-12-02-161113.gpx,0,Cross-Country Skiing 12/2/19 4:11 pm,2019-12-02 15:11:13+00:00,4250.317829,2222,4.784425,0,4250.317829,2019-12-02 15:47:35+00:00,60.200127,12.006074,202.3,1.454602
152745,335,2019-12-02-161113.gpx,0,Cross-Country Skiing 12/2/19 4:11 pm,2019-12-02 15:11:13+00:00,4250.317829,2222,4.784425,0,4250.317829,2019-12-02 15:47:42+00:00,60.200049,12.006139,202.2,1.235034
152746,335,2019-12-02-161113.gpx,0,Cross-Country Skiing 12/2/19 4:11 pm,2019-12-02 15:11:13+00:00,4250.317829,2222,4.784425,0,4250.317829,2019-12-02 15:47:51+00:00,60.199978,12.006254,202.1,0.734641
152747,335,2019-12-02-161113.gpx,0,Cross-Country Skiing 12/2/19 4:11 pm,2019-12-02 15:11:13+00:00,4250.317829,2222,4.784425,0,4250.317829,2019-12-02 15:48:15+00:00,60.199912,12.006320,202.0,0.341874


### Lager kart for alle turene

In [5]:
Start = 0
Stopp = 336

data_kart = df[['File_Index','Point_Latitude','Point_Longitude']]
kart = folium.Map(tiles = "cartodbpositron")
for Tur_nr in range(Start,Stopp):
    
    data_kart = df.loc[df['File_Index'] == Tur_nr]
    data_kart = [[row['Point_Latitude'],row['Point_Longitude']] for i, row in data_kart.iterrows()]
    folium.PolyLine(data_kart, radius=10).add_to(kart)
    
# Må kartet som html for å kunne vise det:
kart.save('kartet_mitt.html')

# Viser kart
#kart

### Test med farger

https://medium.com/@madhuramiah/geographic-plotting-with-python-folium-2f235cc167b7

Frisk opp hvordan lambda fungerer

Ideer til farger jeg kan velge (obs. ikke velg hvit (synes ikke):

https://betterfigures.org/2015/06/23/picking-a-colour-scale-for-scientific-graphics/

In [17]:

# farge = df.loc[df['Segment_Length'] > 10000] 

# farge

Start = 334
Stopp = 336

data_kart = df[['File_Index','Point_Latitude','Point_Longitude']]
kart = folium.Map(tiles = "cartodbpositron")

for Tur_nr in range(Start,Stopp):
    
    data_kart = df.loc[df['File_Index'] == Tur_nr]
#     farge = df.loc[df['Segment_Length'] > 10000] #else farge = 'red'

#     blå = data_kart.loc[data_kart['Segment_Length'] > 9000]
#     rød = data_kart.loc[data_kart['Segment_Length'] > 8000]

#     if data_kart.loc[data_kart['Segment_Length'] == 4250.317829]:
#         farge = 'pink'
#     elif data_kart.loc[data_kart['Segment_Length'] != 4250.317829]:
#         farge = 'blue'

#     if Tur_nr == 334:
#         farge = 'blue'
#     elif Tur_nr == 335:
#         farge = 'red'
        
    if Tur_nr == 334:
        farge = '#08306b'
    elif Tur_nr == 335:
        farge = '#4292c6'
        
#     if any(data_kart.loc[data_kart['Segment_Length'] > 10000]):
#         farge = 'black'
#     elif any(data_kart.loc[data_kart['Segment_Length'] < 5000]):
#         farge = 'red'
    
# farge
# #     data_kart.apply(lambda color:'red' if )
# #Segment_Length
# #     .apply(lambda count:"Black" if count>=400 else
# #                                          "green" if count>=300 and count<400 else
# #                                          "Orange" if count>=200 and count<300 else
# #                                          "darkblue" if count>=150 and count<200 else
# #                                          "red" if count>=100 and count<150 else
# #                                          "lightblue" if count>=75 and count<100 else
# #                                          "brown" if count>=50 and count<75 else
# #                                          "grey")
    data_kart = [[row['Point_Latitude'],row['Point_Longitude']] for i, row in data_kart.iterrows()]
    folium.PolyLine(data_kart, color=farge, radius=10).add_to(kart)
    
# # Må kartet som html for å kunne vise det:
# # kart.save('kartet_mitt.html')

# # Viser kart
kart

### Lager heatmap for alle turene

In [6]:
Start = 0
Stopp = 336

data_heatmap = df[['File_Index','Point_Latitude','Point_Longitude']]
kart_heat = folium.Map(tiles = "cartodbpositron")
for Tur_nr in range(Start,Stopp):
    
    data_heatmap = df.loc[df['File_Index'] == Tur_nr]
    data_heatmap = [[row['Point_Latitude'],row['Point_Longitude']] for i, row in data_heatmap.iterrows()]
    HeatMap(data_heatmap, radius=10).add_to(kart_heat)
    
# Må kartet som html for å kunne vise det:
kart_heat.save('kartet_mitt_heat.html')

# Viser kart
#kart_heat